In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, ShuffleSplit
import numpy as np
from itertools import combinations
import statsmodels.api as sm
import statsmodels.tools as st
from statsmodels.stats.outliers_influence import variance_inflation_factor
import time
import matplotlib.pyplot as plt
import pprint
import statistics
import time
from statsmodels.stats.outliers_influence import variance_inflation_factor
%matplotlib inline

In [2]:
start = time.time()
# response = 'rheomix final deg time min'
response = 'rheomix stability time min'

In [3]:
file1 = 'Total_model_subset.csv'
df_model  = pd.read_csv(file1)
col_list = ['AIC', 'AICc', 'BIC', 'Cond_No']
for i in col_list:
    df_model[i] = ''
print(len(df_model))

2350601


In [4]:
def import_data(experimental_file, response_file, response):
    'imports data'
    'experimental_file = experimental design csv filename i.e experimental.csv'
    'response_file = results csv filename i.e Response.csv'
    'response = reponse name: i.e rheomix final deg time min or rheomix stability time min'
    
    experimental_df = pd.read_csv(experimental_file)
    response_df = pd.read_csv(response_file)
    
    X = experimental_df[experimental_df.columns.values.tolist()[1:]].values
    y = response_df[response].values
    max1 = max(y)
    min1 = min(y)

    y_norm = [2*((i-min1)/(max1-min1)) - 1 for i in y]
    
    
    X_linear = X
    linear_terms = experimental_df.columns.values.tolist()[1:]
    
    return y_norm, X_linear, linear_terms, experimental_df, response_df

def X_gen(model, X_linear):
    
    for i, j in enumerate(model):
        
        if i == 0 and len(j) == 2: 
            X_new = X_linear[:, model[0][1]]
        
        else:
            
            if len(j) == 2:
                add = X_linear[:, j[1]]

            if len(j) == 3:
                add = X_linear[:, j[1]]*X_linear[:, j[2]]

            X_new2 = np.column_stack((X_new, add))
            X_new = X_new2

    
    return X_new

In [5]:
dict1 = {'Xpvc': 0, 'Xfiller': 1, 'Xfr': 2, 'Xstabiliser': 3, 'Xdinp': 4, 'Xldh':5, "Xsph": 6}

experimental_file = 'experimental.csv'
response_file = 'Response.csv'

y, X_linear, linear_terms, experimental_df, response_df = import_data(experimental_file, response_file, response)

models = np.array(df_model['Model'])
list_AIC = []
list_AICc = []
list_BIC = []
list_CN = []

for model in models:
    
    cnt = 0                  
    a = model.split('--')[1:-1]
    i = []
    for b in a:
        if len(b.split('*')) == 1:
            i.append([b, dict1[b]])
        else:
            i.append([b, dict1[b.split('*')[0]], dict1[b.split('*')[1]]])
            
    X = X_gen(i, X_linear)
    
    model_fit = sm.OLS(y, X)
    results = model_fit.fit()
    
    list_AIC.append(results.aic)
    list_BIC.append(results.bic)
    list_AICc.append(st.eval_measures.aicc(results.llf, results.nobs, results.df_model))
    list_CN.append(results.condition_number)
    
col_list = ['AIC', 'AICc', 'BIC', 'Cond_No']
col_list2 = [list_AIC, list_AICc, list_BIC, list_CN]

for i, j in zip(col_list, col_list2):
    df_model[i] = j
    
df_model.to_csv('Total_model_subset_{}_AIC_BIC_AICc.csv'.format(response))

In [6]:
end = time.time()
print(end - start)

2809.596301317215
